In [1]:
from collections import Counter, defaultdict

import numpy as np

import src.util as util
import src.svm as svm

In [2]:
train_messages, train_labels = util.load_spam_dataset("./data/ds6_train.tsv")
val_messages, val_labels = util.load_spam_dataset("./data/ds6_val.tsv")
test_messages, test_labels = util.load_spam_dataset("./data/ds6_test.tsv")

### (a) pre-process message


In [3]:
def get_words(message):
    """Get the normalized list of words from a message string.

    This function should split a message into words, normalize them, and return
    the resulting list. For splitting, you should split on spaces. For normalization,
    you should convert everything to lowercase.

    Args:
        message: A string containing an SMS message

    Returns:
       The list of normalized words from the message.
    """

    # *** START CODE HERE ***
    return message.lower().split(" ")
    # *** END CODE HERE ***


def create_dictionary(messages):
    """Create a dictionary mapping words to integer indices.

    This function should create a dictionary of word to indices using the provided
    training messages. Use get_words to process each message.

    Rare words are often not useful for modeling. Please only add words to the dictionary
    if they occur in at least five messages.

    Args:
        messages: A list of strings containing SMS messages

    Returns:
        A python dict mapping words to integers.
    """

    # *** START CODE HERE ***
    word_dict = defaultdict(int)
    for message in messages:
        for word in get_words(message):
            word_dict[word] += 1

    word_dict = {
        word: index
        for index, word in enumerate(
            word for word, count in word_dict.items() if count >= 5
        )
    }
    return word_dict
    # *** END CODE HERE ***


def transform_text(messages, word_dictionary):
    """Transform a list of text messages into a numpy array for further processing.

    This function should create a numpy array that contains the number of times each word
    appears in each message. Each row in the resulting array should correspond to each
    message and each column should correspond to a word.

    Use the provided word dictionary to map words to column indices. Ignore words that
    are not present in the dictionary. Use get_words to get the words for a message.

    Args:
        messages: A list of strings where each string is an SMS message.
        word_dictionary: A python dict mapping words to integers.

    Returns:
        A numpy array marking the words present in each message.
    """
    # *** START CODE HERE ***
    result = np.zeros((len(messages), len(word_dictionary)))
    for i, message in enumerate(messages):
        word_counts = Counter(get_words(message))
        for word, count in word_counts.items():
            if word in word_dictionary:
                result[i, word_dictionary[word]] = count
    return result
    # *** END CODE HERE ***

In [4]:
dictionary = create_dictionary(train_messages)
train_matrix = transform_text(train_messages, dictionary)
train_matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### (b)


In [7]:
def fit_naive_bayes_model(matrix, labels, k=1):
    """Fit a naive bayes model.
    Note: ham = 0 , spam = 1

    This function should fit a Naive Bayes model given a training matrix and labels.

    The function should return the state of that model.

    Feel free to use whatever datatype you wish for the state of the model.

    Args:
        matrix: A numpy array containing word counts for the training data
        labels: The binary (0 or 1) labels for that training data

    Returns: The trained model
    """
    # *** START CODE HERE ***
    prior_ham = np.mean(labels == 0)
    prior_spam = np.mean(labels == 1)

    model = {"prior_ham": np.log(prior_ham), "prior_spam": np.log(prior_spam)}

    ham_messages = matrix[labels == 0]
    spam_messages = matrix[labels == 1]

    total_ham_word = np.sum(ham_messages)
    total_spam_word = np.sum(spam_messages)

    V = matrix.shape[1]

    # laplace smoothing
    model["ham_probs"] = (ham_messages.sum(axis=0) + k) / (total_ham_word + k * V)
    model["spam_probs"] = (spam_messages.sum(axis=0) + k) / (total_spam_word + k * V)

    return model
    # *** END CODE HERE ***


def predict_from_naive_bayes_model(model, matrix):
    """Use a Naive Bayes model to compute predictions for a target matrix.

    This function should be able to predict on the models that fit_naive_bayes_model
    outputs.

    Args:
        model: A trained model from fit_naive_bayes_model
        matrix: A numpy array containing word counts

    Returns: A numpy array containg the predictions from the model
    """
    # *** START CODE HERE ***
    ham_probs_log = np.log(model["ham_probs"])
    spam_probs_log = np.log(model["spam_probs"])

    ham_scores = matrix.dot(ham_probs_log) + model["prior_ham"]
    spam_scores = matrix.dot(spam_probs_log) + model["prior_spam"]

    return (ham_scores < spam_scores).astype(int)
    # *** END CODE HERE ***

In [8]:
val_matrix = transform_text(val_messages, dictionary)
test_matrix = transform_text(test_messages, dictionary)

naive_bayes_model = fit_naive_bayes_model(train_matrix, train_labels, k=5)

naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, test_matrix)
naive_bayes_accuracy = np.mean(naive_bayes_predictions == test_labels)
print("test accuracy: ", naive_bayes_accuracy)

naive_bayes_predictions = predict_from_naive_bayes_model(naive_bayes_model, val_matrix)
naive_bayes_accuracy = np.mean(naive_bayes_predictions == val_labels)
print("validation accuracy: ", naive_bayes_accuracy)

test accuracy:  0.9802867383512545
validation accuracy:  0.9820466786355476


### (c)


In [29]:
def get_top_five_naive_bayes_words(model, dictionary):
    """Compute the top five words that are most indicative of the spam (i.e positive) class.

    Ues the metric given in 6c as a measure of how indicative a word is.
    Return the words in sorted form, with the most indicative word first.

    Args:
        model: The Naive Bayes model returned from fit_naive_bayes_model
        dictionary: A mapping of word to integer ids

    Returns: The top five most indicative words in sorted order with the most indicative first
    """
    # *** START CODE HERE ***
    token_given_ham = model["ham_probs"]
    token_given_spam = model["spam_probs"]
    token_scores = np.log(token_given_spam) - np.log(token_given_ham)
    top_5_indices = token_scores.argsort()[-5:][::-1]

    top_5_words = [
        (k, token_scores[v]) for k, v in dictionary.items() if v in top_5_indices
    ]
    sorted_top_5_words = sorted(top_5_words, key=lambda x: -x[1])
    print(sorted_top_5_words)
    # *** END CODE HERE ***


get_top_five_naive_bayes_words(naive_bayes_model, dictionary)

[('claim', 3.8572617067628734), ('won', 3.291947897712813), ('prize', 3.2717451903952934), ('now!', 3.1705870407085452), ('tone', 3.017511052011053)]


### (d)


In [38]:
def compute_best_svm_radius(
    train_matrix, train_labels, val_matrix, val_labels, radius_to_consider
):
    """Compute the optimal SVM radius using the provided training and evaluation datasets.

    You should only consider radius values within the radius_to_consider list.
    You should use accuracy as a metric for comparing the different radius values.

    Args:
        train_matrix: The word counts for the training data
        train_labels: The spma or not spam labels for the training data
        val_matrix: The word counts for the validation data
        val_labels: The spam or not spam labels for the validation data
        radius_to_consider: The radius values to consider

    Returns:
        The best radius which maximizes SVM accuracy.
    """
    # *** START CODE HERE ***
    res = []
    for rad in radius_to_consider:
        svm_predictions = svm.train_and_predict_svm(
            train_matrix, train_labels, val_matrix, rad
        )
        res.append((rad, np.mean(svm_predictions == val_labels)))

    optimal_radius = sorted(res, key=lambda x: -x[1])
    return optimal_radius[0][0]
    # *** END CODE HERE ***

In [40]:
optimal_radius = compute_best_svm_radius(
    train_matrix, train_labels, val_matrix, val_labels, [0.01, 0.1, 1, 10]
)
print("The optimal SVM radius was {}".format(optimal_radius))

svm_predictions = svm.train_and_predict_svm(
    train_matrix, train_labels, test_matrix, optimal_radius
)

svm_accuracy = np.mean(svm_predictions == test_labels)

print(
    "The SVM model had an accuracy of {} on the testing set".format(
        svm_accuracy, optimal_radius
    )
)

The optimal SVM radius was 0.1
The SVM model had an accuracy of 0.9713261648745519 on the testing set
